# General

## Importing Libraries and Modules

In [15]:
import pandas as pd
import matplotlib as plt

import pickle

from sqlalchemy import create_engine

import etl.extract.tb as tb_proc 
import etl.extract.wos as wos_parser

## CRISP-DM

## ETL

### (E)xtraction


#### Web of Science/Zoological Records
WoS do not provide an API, as far as I know, and has strict access policies across the board of all of their products. Thus, I downloaded the search results using their GUI, manually, and coded a parser from scratch that loads the data from the 150+ files into a SQLite database.

The specifics on how I queried this system is in the report, Methods section.

In [2]:
# Parse data, creates df
wos = wos_parser.create_df()

# This is commented out because it generates a over 200Mb file that I couldn't push to GitHub, but I left it commented off here to show that I've worked on this

#wos_parser.to_pickle(wos, 'data/wos.p')

# As an last minute alternative, I'm using this command here to save the pandas.DataFrame into a SQLite database

#TODO I've to work on this.

In [3]:
# Checking if it loaded
wos.head()

,Publication Type,Zoological Record Accession Number,Document Type,Title,Foreign Title,Authors,Source,Volume,Page Span,Publication Date,...,Journal URL,Item URL,Notes,NaN,Book Authors,Publisher,Publisher Address,International Standard Book Number (ISBN),Editors,Group Authors
0,J,ZOOR14707048981,Article,Orthopterans of the Reserva Biologica Alberto ...,Ortopteros de la Reserva Biologica Alberto Man...,"Barranco, Pablo (pbvega@ual.es)",Boletin de la SEA,47,21-32,31 Dec 2010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,J,ZOOR14707048982,Article,Further considerations on the genus Ananteris ...,NaN,"Lourenco, Wilson R. (arache@mnhn.fr), Duhem, B...",Boletin de la SEA,47,33-38,31 Dec 2010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,J,ZOOR14707048983,Article,Description of a new species of the genus Mela...,Descripcion de una nueva especie del genero Me...,"Ferrer, Julio, Tovar, Alejandro Castro",Boletin de la SEA,47,39-44,31 Dec 2010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,J,ZOOR14707048985,Article,New arachnids from Puerto Rico (Arachnida: Amb...,Nuevos aracnidos de Puerto Rico (Arachnida: Am...,"de Armas, Luis F. (zoologia.ies@ama.cu)",Boletin de la SEA,47,55-64,31 Dec 2010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,J,ZOOR14707048986,Article,Description of new hypogean taxa of Laparoceru...,Descripcion de nuevos Laparocerus hipogeos de ...,"Machado, Antonio (a.machado@telefonica.net), G...",Boletin de la SEA,47,65-69,31 Dec 2010,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### TreatmentBank (Plazi)
TreatmentBank has a powerful API with GUI. I used it to built the target query, and copied/pasted their URL into my code. From there I processed the results, adding them into a SQLite database. 

In [6]:
# Process the hard-coded API call and creates a database from it
# I'm commenting out because the SQLite db was provided and it contains a snapshot of the data I analyzed. As this includes an API call, running this now might get new data, changing my results

#tb_proc.create_db(tb_proc.get_data())

# Alternatively, I can simply load the .db
db_connect = create_engine('sqlite:///data/tb.db')

tb = pd.read_sql("docs", con=db_connect)

In [7]:
# Checking if it loaded
tb.head()

,id,authors,year,title,journal,volume,issue,start_page,end_page,num_treats
0,1,"Fernandez-Triana, Jose L & Whitfield, James B ...",2014,First record of the genus Venanus (Hymenoptera...,Biodiversity Data Journal,2,0,4167,4167,4
1,2,"Barrales-Alcala, Diego A. & Francke, Oscar F.",2018,A new Sky Island species of Vaejovis C. L. Koc...,ZooKeys,760,0,37,53,2
2,3,"Guevara-Guerrero, Gonzalo & Bonito, Gregory & ...",2018,"Tuberaztecorum sp. nov., a truffle species fro...",MycoKeys,30,0,61,72,2
3,4,"Balke, Michael & Ruthensteiner, Bernhard & War...",2015,Two new species of Limbodessus diving beetles ...,Biodiversity Data Journal,3,0,7096,7096,4
4,5,"Costa, Wilson J. E. M. & Amorim, Pedro F.",2018,Cryptic species diversity in the Hypsolebiasma...,ZooKeys,777,0,141,158,4


#### Zoobank
As explained, I performed a web scraping to collect data from Zoobank. For this, I used a Scrapy project, with custom Item, ItemLoader, in- and output processors and ItemPipeline. These can't quite be run from here, as far as I know, I can only create a CrawlerProcess that won't have these functionalities. This was then saved into a SQLite database, which is alternatively loaded here.

In [8]:
# Load the .db created from the web scraping
db_connect = create_engine('sqlite:///data/zb.db')

zb = pd.read_sql("docs", con=db_connect)

In [9]:
# Checking if it loaded
zb.head()

,id,authors,year,title,journal,bibref_details,volume,issue,start_page,end_page,from_year
0,1,"Achatz, Johannes G., Matthew D. Hooge, A. Wall...",2010,Systematic revision of acoels with 9+0 sperm u...,Journal of Zoological Systematics and Evolutio...,48(1): 9-32,48,1,9,32,2010
1,2,"Affilastro, Andrés A. O. & Ignacio Garcia-Mauro.",2010,"A new Bothriurus (Scorpiones, Bothriuridae) fr...",Zootaxa,2488: 52-64,2488,,52,64,2010
2,3,"Ahyong, Shane T.",2010,The marine fauna of New Zealand: king crabs of...,No,,,,,,2010
3,4,"Ahyong, Shane T., Keiji Baba, Enrique Macphers...",2010,A new classification of the Galatheoidea (Crus...,Zootaxa,2676: 57-68,2676,,57,68,2010
4,5,"Aibek, Ulykpan & Seiki Yamane.",2010,Discovery of the Subgenera Austrolasius and De...,Japanese Journal of systematic entomology,16(2): 197-202,16,2,197,202,2010


#### Exploratory Analyses: Extraction Tests
Here I will make a series of tests to check if I'm getting all the data that I was supposed to be getting. For that, I'm going to use Python's assert, and some manual observations.


##### Web of Science/Zoological Records
These were the observed results from the WoS GUI, per year:

- 2010: 7207 results
- 2011: 7480 results
- 2012: 7604 results
- 2013: 7655 results
- 2014: 7456 results
- 2015: 7983 results
- 2016: 7859 results
- 2017: 8023 results
- 2018: 7973 results
- 2019: 7664 results
- 2020: 3548 results


In [10]:
# Create a dict with year: official results, to be used on a data consistency test
wos_results = {
    '2010': 7207,
    '2011': 7480,
    '2012': 7604,
    '2013': 7655,
    '2014': 7456,
    '2015': 7983,
    '2016': 7859,
    '2017': 8023,
    '2018': 7973,
    '2019': 7664,
    '2020': 3548,
}

In [36]:
# Checkout column names to get the right one: 'Year Published'
wos.columns

Index([                                     'Publication Type',
                          'Zoological Record Accession Number',
                                               'Document Type',
                                                       'Title',
                                               'Foreign Title',
                                                     'Authors',
                                                      'Source',
                                                      'Volume',
                                                   'Page Span',
                                            'Publication Date',
                                              'Year Published',
                                                    'Language',
                                 'Usage Count (Last 180 Days)',
                                    'Usage Count (Since 2013)',
                                                    'Abstract',
                                        

In [12]:
# Create a pandas.Series with the ordered counts of the 'Year Published' column
wos_years_count = wos['Year Published'].value_counts().sort_index()

In [47]:
def test_data(source, series_wos, dict_real_wos):
    """Test unit for the Web of Science/Zoological Records data extraction

        Args:
            source (str): Name of the source
            series_wos (pandas.Series): Contains the value counts per year from the extracted data
            dict_real_wos (dict): Contains the actual observed results from WoS GUI
    """
    wos_total_results = sum(dict_real_wos.values())

    print('-'*74)                   # Begin with my cool divider
    print("Testing for... {}".format(source.upper()))
    print('-'*74)                   # Print a cool divider

    # Assertions about the total counts
    try:
        assert(wos_total_results == series_wos.sum())
        print("SUCCESS! Total count was equal between the processed and official results!")
    except:
        print("FAILED! There is a difference of {} records between the processed and official results!".format(series_wos.sum() - wos_total_results))

    print('-'*74)                   # Print a cool divider

    # # Assertions about the individual years
    for key in wos_results:
        try:
            assert(dict_real_wos[key] == series_wos[key])
            print("SUCCESS! the year {} seems to be fine!".format(key))
        except:
            print("FAILED! The year {} got a difference of {} between the processed and official results".format(key, series_wos[key] - dict_real_wos[key]))
    
    print('-'*74)                   # Finish with my cool divider



In [48]:
# Calling the test with WoS/Zoological Record data
test_data("WoS/Zoological Records", wos_years_count, wos_results)

--------------------------------------------------------------------------
Testing for... WOS/ZOOLOGICAL RECORDS
--------------------------------------------------------------------------
SUCCESS! Total count was equal between the processed and official results!
--------------------------------------------------------------------------
SUCCESS! the year 2010 seems to be fine!
SUCCESS! the year 2011 seems to be fine!
SUCCESS! the year 2012 seems to be fine!
SUCCESS! the year 2013 seems to be fine!
SUCCESS! the year 2014 seems to be fine!
SUCCESS! the year 2015 seems to be fine!
SUCCESS! the year 2016 seems to be fine!
SUCCESS! the year 2017 seems to be fine!
SUCCESS! the year 2018 seems to be fine!
SUCCESS! the year 2019 seems to be fine!
SUCCESS! the year 2020 seems to be fine!
--------------------------------------------------------------------------


##### TreatmentBank
I can't make new API calls. The reason why is that as the data is live I might risk to get more data than I have got when I first run this, which could cause different results. 

That's why I saved a .pickle file with the entire response of the API on the routine call, besides the .db resulting of the processing of this data. As such, I can compare the total number of records in both (and per year!) to check if my processing missed something or not, using the same function applied to WoS data.

In [43]:
# Creating a dict with the counts of records per year, based on the total response saved from the API call to TreatmentBank

# Open .pickle
with open('data/tb_response.p', 'rb') as p_file:
        tb_response = pickle.load(p_file)

tb_response_years_count = dict()

# Iterate over the full response, and create the counting
for result in tb_response['data']:
    try:
        tb_response_years_count[result['BibYear']] += 1
    except:
        tb_response_years_count[result['BibYear']] = 1

In [46]:
# Create a pandas.Series with the ordered counts of the 'year' column
tb_years_count = tb['year'].value_counts().sort_index()

In [49]:
# Calling the test with TreatmentBank data
test_data("TreatmentBank", tb_years_count, tb_response_years_count)

--------------------------------------------------------------------------
Testing for... TREATMENTBANK
--------------------------------------------------------------------------
SUCCESS! Total count was equal between the processed and official results!
--------------------------------------------------------------------------
SUCCESS! the year 2010 seems to be fine!
SUCCESS! the year 2011 seems to be fine!
SUCCESS! the year 2012 seems to be fine!
SUCCESS! the year 2013 seems to be fine!
SUCCESS! the year 2014 seems to be fine!
SUCCESS! the year 2015 seems to be fine!
SUCCESS! the year 2016 seems to be fine!
SUCCESS! the year 2017 seems to be fine!
SUCCESS! the year 2018 seems to be fine!
SUCCESS! the year 2019 seems to be fine!
SUCCESS! the year 2020 seems to be fine!
--------------------------------------------------------------------------


##### Zoobank
These were the observed results from the Zoobank, per year:

- 2010: 1128 results
- 2011: 1430 results
- 2012: 2482 results
- 2013: 4293 results
- 2014: 4829 results
- 2015: 5552 results
- 2016: 5732 results
- 2017: 6098 results
- 2018: 6401 results
- 2019: 6386 results
- 2020: 4554 results

In [59]:
# Create a dict with year: official results, to be used on a data consistency test
zb_results = {
    '2010': 1128,
    '2011': 1430,
    '2012': 2482,
    '2013': 4293,
    '2014': 4829,
    '2015': 5552,
    '2016': 5732,
    '2017': 6098,
    '2018': 6401,
    '2019': 6386,
    '2020': 4554,
}

In [54]:
# Check the existing columns in this dataframe
list(zb.columns)

['id',
 'authors',
 'year',
 'title',
 'journal',
 'bibref_details',
 'volume',
 'issue',
 'start_page',
 'end_page',
 'from_year']

Here there is one interesting caveat. Zoobank allow users to register papers from the past, and when queried by year, it shows not the publications published on that year, but the publications **registred** on that year. 

As such, we might have all sort of years in the column 'year'.

In [56]:
# Create a pandas.Series with the ordered counts of the 'year' column
zb_years_count = zb['year'].value_counts().sort_index()

# Check results
zb_years_count

1847       2
1877       2
1878       1
1880       2
1881       1
1883       1
1887       2
1888       2
1889       3
1894       1
1896       2
1898       2
1902       1
1905       1
1917       1
1919       3
1923       1
1927       3
1932       1
1933       1
1936       1
1937       5
1951       4
1952       3
1960       1
1962       1
1967       1
1969       1
1972       2
1986       1
1989       1
1996       2
1999       4
2001       1
2002       1
2003       9
2004       1
2008       1
2009       4
2010    1023
2011    1318
2012    2387
2013    4253
2014    4774
2015    5550
2016    5779
2017    6119
2018    6434
2019    6497
2020    4674
Name: year, dtype: int64

To address this situation, I included a column on zb.db called 'from_year', which extracts the year being queried on the response.url, inside the parser() of the spider used to scrape Zoobank, and add to this column. 

This means that we can get the total (included) per year, to compare with the official results.

In [57]:
# Create a pandas.Series with the ordered counts of the 'Year Published' column
zb_years_count = zb['from_year'].value_counts().sort_index()

# Check results
zb_years_count

2010    1128
2011    1430
2012    2482
2013    4293
2014    4829
2015    5552
2016    5732
2017    6098
2018    6401
2019    6386
2020    4554
Name: from_year, dtype: int64

In [60]:
# Now that I have both dictionary with official results and the pandas.Series with the processed results, I can call the unit test function
test_data("Zoobank", zb_years_count, zb_results) 

--------------------------------------------------------------------------
Testing for... ZOOBANK
--------------------------------------------------------------------------
SUCCESS! Total count was equal between the processed and official results!
--------------------------------------------------------------------------
SUCCESS! the year 2010 seems to be fine!
SUCCESS! the year 2011 seems to be fine!
SUCCESS! the year 2012 seems to be fine!
SUCCESS! the year 2013 seems to be fine!
SUCCESS! the year 2014 seems to be fine!
SUCCESS! the year 2015 seems to be fine!
SUCCESS! the year 2016 seems to be fine!
SUCCESS! the year 2017 seems to be fine!
SUCCESS! the year 2018 seems to be fine!
SUCCESS! the year 2019 seems to be fine!
SUCCESS! the year 2020 seems to be fine!
--------------------------------------------------------------------------


It seems that the data I processed into databases are consistent with the data observed from the sources. The processing stages were done flawlessly. 

It's time to move on.

### (T)ransform


#### Web of Science/Zoological Records


#### TreatmentBank (Plazi)


#### Zoobank


#### SHERPA/RoMEO
At this point, I'm not sure if I'll include it.


#### Exploratory Analyses: Transform


### (L)oad


## Analysis


### Exploratory Analyses: Final


### Question #1


### Question #2


### Question #3

### Question #4

### Question #5

## Conclusion

## References
I'm not sure if i'll maintain this section or only add these to the README.md